In [ ]:
using Gmsh: gmsh
using Gridap
using GridapGmsh
using Gridap.TensorValues
using LineSearches: BackTracking
using Gridap.Arrays
using Gridap.ReferenceFEs
using Plots
using LinearAlgebra

In [ ]:
I2 = SymTensorValue{2,Float64}(1.0 ,0.0 ,1.0)
I4 = I2⊗I2
I4_sym = one(SymFourthOrderTensorValue{2,Float64})
I4_vol = (1.0/2)*I4
I4_dev = I4_sym  - I4_vol

In [ ]:
const ls = 1.0

In [ ]:
const L = 102
const H = 102
const a = 20.32
const c = 0.5*a
const θ = pi/4

const CH = 0.204
const hfc = ls/2.1
const hf = ls/2.1
const h = 10*hf 
const FMR = 5*0.5

#Coordinates
A1 = 0.5*L - c*cos(θ) - CH*sin(θ)
A2 = 0.5*H + c*sin(θ) - CH*cos(θ)

A11 = 0.5*L - c*cos(θ) + CH*sin(θ)
A22 = 0.5*H + c*sin(θ) + CH*cos(θ)

B1 = 0.5*L - (c+a)*cos(θ) - CH*sin(θ)
B2 = 0.5*H + (c+a)*sin(θ) - CH*cos(θ)

B11 = 0.5*L - (c+a)*cos(θ) + CH*sin(θ)
B22 = 0.5*H + (c+a)*sin(θ) + CH*cos(θ)

C1 = 0.5*L + c*cos(θ) - CH*sin(θ)
C2 = 0.5*H - c*sin(θ) - CH*cos(θ)

C11 = 0.5*L + c*cos(θ) + CH*sin(θ)
C22 = 0.5*H - c*sin(θ) + CH*cos(θ)

D1 = 0.5*L + (c+a)*cos(θ) - CH*sin(θ)
D2 = 0.5*H - (c+a)*sin(θ) - CH*cos(θ)

D11 = 0.5*L + (c+a)*cos(θ) + CH*sin(θ)
D22 = 0.5*H - (c+a)*sin(θ) + CH*cos(θ)

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.geo.addPoint(0.0, 0.0, 0.0, h, 1)  
gmsh.model.geo.addPoint(L, 0.0, 0.0, h, 2) 
gmsh.model.geo.addPoint(L, H, 0.0, h, 3) 
gmsh.model.geo.addPoint(0.0, H, 0.0, h, 4)

gmsh.model.geo.addPoint(A1, A2, 0.0, h, 5)
gmsh.model.geo.addPoint(A11, A22, 0.0, h, 6)
gmsh.model.geo.addPoint(B11, B22, 0.0, h, 7)
gmsh.model.geo.addPoint(B1, B2, 0.0, h, 8)

gmsh.model.geo.addPoint(C1, C2, 0.0, h, 9)
gmsh.model.geo.addPoint(C11, C22, 0.0, h, 10)
gmsh.model.geo.addPoint(D11, D22, 0.0, h, 11)
gmsh.model.geo.addPoint(D1, D2, 0.0, h, 12)

gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 1, 4)

gmsh.model.geo.addLine(5, 6, 5)
gmsh.model.geo.addLine(6, 7, 6)
gmsh.model.geo.addLine(7, 8, 7)
gmsh.model.geo.addLine(8, 5, 8)

gmsh.model.geo.addLine(9, 10, 9)
gmsh.model.geo.addLine(10, 11, 10)
gmsh.model.geo.addLine(11, 12, 11)
gmsh.model.geo.addLine(12, 9, 12)

gmsh.model.geo.addLine(1, 3, 55)

gmsh.model.geo.addCurveLoop([1,2,3,4],1) 
gmsh.model.geo.addCurveLoop([5,6,7,8],2) 
gmsh.model.geo.addCurveLoop([9,10,11,12],3) 

gmsh.model.geo.addPlaneSurface([1,-2, -3], 1)

gmsh.model.addPhysicalGroup(2, [1],1)

gmsh.model.addPhysicalGroup(1, [1],1)
gmsh.model.addPhysicalGroup(1, [3],2)

gmsh.model.setPhysicalName(2, 1, "Domain")
gmsh.model.setPhysicalName(1, 1, "DirichletBot")
gmsh.model.setPhysicalName(1, 2, "DirichletTop")


gmsh.model.mesh.field.add("Distance", 1)
gmsh.model.mesh.field.setNumbers(1, "EdgesList", [55])

gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "IField", 1)
gmsh.model.mesh.field.setNumber(2, "LcMin", hf)
gmsh.model.mesh.field.setNumber(2, "LcMax", h)
gmsh.model.mesh.field.setNumber(2, "DistMin", 30*FMR)
gmsh.model.mesh.field.setNumber(2, "DistMax", 32*FMR)

gmsh.model.mesh.field.setAsBackgroundMesh(2)

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("SquarePlateWithEdgeNotch.msh")
gmsh.finalize()

In [ ]:
model = GmshDiscreteModel("SquarePlateWithEdgeNotch.msh")
writevtk(model,"SquarePlateWithEdgeNotch")

In [ ]:
using Gridap.Geometry
labels = get_face_labeling(model)
dimension = 2
mat_tags = get_face_tag(labels,dimension)

In [ ]:
const E = 36.2e3
const ν = 0.21
const G = E/(2*(1+ν))
const l = 15.0
const N = 0.3
const Gc = 0.011

const η = 1e-15

In [ ]:
const λ = 2*G*ν/(1 -2*ν)
const κ = 2*G*N^2/(1-N^2)
const μ = G*(1-2*(N^2))/(1-N^2)
const γ = 4*G*l^2

In [ ]:
const σc = 10 
const m = 3*E*Gc/(4*ls*(σc^2))
const ψ_Crit = σc^2 / (2*E)
const kf = 0.5
const αT = ψ_Crit/kf

In [ ]:
path = "./SENT_DoubleInclined_lb15_N$N/"                                # Output path
mkpath(path) 
cd("SENT_DoubleInclined_lb15_N$N/")

In [ ]:
function degDer(ϕ)
    g = (m*ϕ)/(0.25*m^2*ϕ^4 - 0.5*m^2*ϕ^2 + 0.25*m^2 - m*ϕ^4 + m*ϕ^2 + ϕ^4) 
    return g
end

In [ ]:
function σ_Bmod(ε, ε_in, s_in)
    gs = (s_in^2) / (s_in^2 + m*(1-s_in)*(0.5+0.5*s_in))
    if tr(ε_in) >= 0
        σM = (gs)*((λ + μ + (κ/2))*tr(ε)*one(ε) + (2*μ + κ)*(I4_dev ⊙ ε))
    else
        σM = (gs)*((2*μ + κ)*(I4_dev ⊙ ε)) + (λ + μ + (κ/2))*tr(ε)*one(ε)
    end
    return σM
end

In [ ]:
E_Matrx = TensorValue(0,1,-1,0)

In [ ]:
function ε_Skw(∇,θ)
    ∇ᵀ = transpose(∇)
    w = (0.5*(∇ᵀ - ∇)) - (E_Matrx*θ)
    return w
end

In [ ]:
function σ_Cmod(ϵ_skew, s_in)
    gs = (s_in^2) / (s_in^2 + m*(1-s_in)*(0.5+0.5*s_in))
    σM = (gs)*(κ*ϵ_skew)
    return σM
end

In [ ]:
function M_mod(∇, s_in)
    gs = (s_in^2) / (s_in^2 + m*(1-s_in)*(0.5+0.5*s_in))
    M = (gs)*(γ*∇)
    return M
end

In [ ]:
function ψPos(ε_in,∇_uh,θ, ∇_th,Fdg)
    ∇_uhᵀ = transpose(∇_uh)
    ϵ_skew = 0.5*(∇_uhᵀ - ∇_uh) - E_Matrx*θ
    if tr(ε_in) >= 0
        ψPos = (0.5*((λ + μ + (κ/2))*(tr(ε_in))^2 + (2*μ+κ)*((I4_dev ⊙ ε_in) ⊙ (I4_dev ⊙ ε_in))) + 0.5*(κ*(ϵ_skew ⊙ ϵ_skew)) + 0.5*γ*(∇_th ⋅ ∇_th))/Fdg
    else
        ψPos = (0.5*((2*μ+κ)*((I4_dev ⊙ ε_in) ⊙ (I4_dev ⊙ ε_in))) + 0.5*(κ*(ϵ_skew ⊙ ϵ_skew)) + 0.5*γ*(∇_th ⋅ ∇_th))/Fdg
    end
    return ψPos 
end

function ψPosOrg(ε_in,∇_uh,θ, ∇_th)
    ∇_uhᵀ = transpose(∇_uh)
    ϵ_skew = 0.5*(∇_uhᵀ - ∇_uh) - E_Matrx*θ
    if tr(ε_in) >= 0
        ψPos = 0.5*((λ + μ + (κ/2))*(tr(ε_in))^2 + (2*μ+κ)*((I4_dev ⊙ ε_in) ⊙ (I4_dev ⊙ ε_in))) + 0.5*(κ*(ϵ_skew ⊙ ϵ_skew)) + 0.5*γ*(∇_th ⋅ ∇_th)
    else
        ψPos = 0.5*((2*μ+κ)*((I4_dev ⊙ ε_in) ⊙ (I4_dev ⊙ ε_in))) + 0.5*(κ*(ϵ_skew ⊙ ϵ_skew)) + 0.5*γ*(∇_th ⋅ ∇_th)
    end
    return ψPos 
end

In [ ]:
function new_EnergyState(ψPlusPrev_in,ψhPos_in)
    ψPlus_in = ψhPos_in
    if ψPlus_in >= ψPlusPrev_in
        ψPlus_out = ψPlus_in
    else
        ψPlus_out = ψPlusPrev_in
    end
    true,ψPlus_out
end

In [ ]:
function project(q,model,dΩ,order)
  reffe = ReferenceFE(lagrangian,Float64,order)
  V = FESpace(model,reffe,conformity=:L2)
  a(u,v) = ∫( u*v )*dΩ
  l(v) = ∫( v*q )*dΩ
  op = AffineFEOperator(a,l,V,V)
  qh = solve(op)
  qh
end

In [ ]:
function FatigueDegrad(αb)
    fdg = @. ifelse(αb >= αT , ((2*αT)/(αb + αT))*((2*αT)/(αb + αT)), 1.0)
    return fdg
end

In [ ]:
function FatiguehistoryVariable(ψhPos_in,αPrev,αbPrev)
    α = ψhPos_in #(s_in ^2 + η)*ψhPos_in
    if α >= αPrev
       αb = αbPrev + abs(α-αPrev)
    else
       αb = αbPrev
    end
return αb
end

In [ ]:
function αNC(s_in,ψhPos_in)
    gs = (s_in^2) / (s_in^2 + m*(1-s_in)*(0.5+0.5*s_in))
    α = gs * ψhPos_in
return α
end

In [ ]:
order = 1
degree = 2*order

In [ ]:
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

In [ ]:
sId = CellState(1.0,dΩ)
shId = project(sId,model,dΩ,order)

In [ ]:
LoadTagId = get_tag_from_name(labels,"DirichletTop")
Γ_Load = BoundaryTriangulation(model,tags = LoadTagId)
dΓ_Load = Measure(Γ_Load,degree)
n_Γ_Load = get_normal_vector(Γ_Load)

In [ ]:
reffe_PF = ReferenceFE(lagrangian,Float64,order)
V0_PF = TestFESpace(model,reffe_PF;conformity=:H1)
sh = zero(V0_PF)

In [ ]:
reffe_theta = ReferenceFE(lagrangian,Float64,order)
V0_theta  = TestFESpace(model,reffe_theta;
  conformity=:H1,dirichlet_tags=["DirichletBot"])
θh = zero(V0_theta)

In [ ]:
reffe_Disp = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
        V0_Disp = TestFESpace(model,reffe_Disp;
          conformity=:H1,
          dirichlet_tags=["DirichletBot","DirichletTop"],
          dirichlet_masks=[(true,true), (false,true)])
uh = zero(V0_Disp)

In [ ]:
V0 = MultiFieldFESpace([V0_Disp,V0_theta])

In [ ]:
function crack_tip_tracker(sh,tol)
    damagetracker = Float64[]
    sVec = Float64[]
    coords = get_node_coordinates(Ω)
    
    sVec = get_free_dof_values(sh)
    
    for i in 1:length(sVec)
        if sVec[i] <= tol
            push!(damagetracker,i)
        end
    end
   damagetracker = round.(Int,damagetracker)
    xloc = Float64[]
    yloc = Float64[]
    if damagetracker!= Float64[]
        for i in 1:length(damagetracker)
            push!(xloc,coords[damagetracker[i]][1])
            push!(yloc,coords[damagetracker[i]][2])
        end
        Locx, index= findmax(xloc)
        Locy = yloc[index]
        return (Locx,Locy)
    else 
       return "No damage"
    end
end

In [ ]:
nls = NLSolver(
  show_trace=true,
  method=:newton,
  linesearch=BackTracking(), iterations = 10)
solver = FESolver(nls)

In [ ]:
function stepPhaseField(x0,vApp,cache,ψPlusPrev_in)
     U_PF = TrialFESpace(V0_PF)
     res(s,ϕ) = ∫( (3/4)*Gc*ls*∇(ϕ)⋅ ∇(s) + (degDer∘(s))*ψPlusPrev_in*ϕ - ((3/8)*Gc/ls)*ϕ)*dΩ
     op = FEOperator(res,U_PF,V0_PF)
     sh_out = FEFunction(U_PF,x0)
     sh_out, cache = solve!(sh_out,solver,op,cache)
  return sh_out, get_free_dof_values(sh_out), cache
end

In [ ]:
function Skw(u,θ)
    ∇ᵀ = transpose(∇(u))
    w = (0.5*(∇ᵀ) - (E_Matrx*θ))
    return w
end

In [ ]:
function   stepDisp(uh_in,θ_in,sh_in,uApp)
        uApp1(x) = VectorValue(0.0,0.0)
        uApp2(x) = VectorValue(0.0,uApp)
        U_Disp = TrialFESpace(V0_Disp,[uApp1,uApp2])
        
        θApp1(x) = 0.0
        U_theta = TrialFESpace(V0_theta,[θApp1])
        U = MultiFieldFESpace([U_Disp,U_theta ])

        a((u,θ),(w,v))  = ∫( ((Skw(w,v)) ⊙ (σ_Bmod∘(ε(u),ε(uh_in),sh_in)) ) + ((Skw(w,v)) ⊙ (σ_Cmod∘(ε_Skw∘(∇(u),θ), sh_in)) ) + ((∇(v))⋅ (M_mod∘(∇(θ),sh_in))) - (v*((E_Matrx) ⊙ (σ_Cmod∘(ε_Skw∘(∇(u),θ),sh_in))) ))*dΩ
        b((w,v))= 0
        op_Disp = AffineFEOperator(a,b,U,V0)
        uh_out = solve(op_Disp)
        uh_out,phih_out = uh_out
    return uh_out, phih_out
end

In [ ]:
vApp = 0
const innerMax = 10
const tol_lim = 1e-10
count = 0

const T = 1
const n = 300
const Tₚ = T/n
const n1 = 16 # loadsteps
const delv = 0.004
const tp = Tₚ/n1
t = 0.0
cycle = 0
const dc = 1/n1

Load = Float64[]
Displacement = Float64[]
time = Float64[]
AppDisplacement = Float64[]
αEnergy = Float64[]
αDegrad = Float64[]
NoofCycles = Float64[]
Xloccrack = Float64[]
Yloccrack = Float64[]
FracEnergy = Float64[]


push!(Xloccrack,(L/2))
push!(Yloccrack,0.5*H)
push!(Load, 0.0)
push!(Displacement, 0.0)

push!(time, t)
push!(AppDisplacement, vApp)
push!(NoofCycles, cycle)

push!(αEnergy, 0.0)
push!(αDegrad, 1.0)
push!(FracEnergy, 0.0)


sPrev = CellState(1.0,dΩ)
sh = project(sPrev ,model ,dΩ,order)
x0_PF = ones(Float64,num_free_dofs(V0_PF))
cache_1 = nothing

αPrev = CellState(0.0,dΩ)
αbPrev = CellState(0.0,dΩ)

αhPlusPrev = project(αPrev,model,dΩ,order)
αbhPlusPrev = project(αbPrev,model,dΩ,order)

FdhPrev = FatigueDegrad∘(αbhPlusPrev)

ψPlusPrev = CellState(ψ_Crit,dΩ)

while t .< T 
        
    t = t + tp
    vAppC = abs(4*delv/Tₚ * abs((((t-Tₚ/4)%Tₚ)+Tₚ)%Tₚ - Tₚ/2) - delv)
    vApp = round(vAppC,digits = 6)
    count = count .+ 1
    cycle = cycle + dc
    FdhPrev = FatigueDegrad∘(αbhPlusPrev)
   
    print("\n Entering displacemtent step$count :", float(vApp))
    
   for inner = 1:innerMax
        
        ψhPlusPrev = project(ψPlusPrev,model,dΩ,order)
        
        RelErr = abs(sum(∫( (3/4)*Gc*ls*∇(sh)⋅ ∇(sh) + (degDer∘(sh))*ψhPlusPrev*sh)*dΩ-∫( ((3/8)*Gc/ls)*sh)*dΩ))/abs(sum(∫( ((3/8)*Gc/ls)*sh)*dΩ))
        print("\n Relative error = ",float(RelErr))
        
        sh,x0_PF,cache_1 = stepPhaseField(x0_PF,vApp,cache_1,ψhPlusPrev) 
        uh,θh = stepDisp(uh,θh,sh,vApp)
        
        ψhPos_in = ψPos∘(ε(uh),∇(uh),θh,∇(θh),FdhPrev)      
        
        update_state!(new_EnergyState,ψPlusPrev,ψhPos_in)
        
        if RelErr < 1e-8
            break 
        end
    end
    
    if  crack_tip_tracker(sh,tol_lim) =="No damage"
        push!(Xloccrack,0.5*L)
        push!(Yloccrack,0.5*H)
    else
        (X_t, Y_t) = crack_tip_tracker(sh,tol_lim)
        push!(Xloccrack,X_t)
        push!(Yloccrack,Y_t)
    end
    
    ψhPosOrg_in = ψPosOrg∘(ε(uh),∇(uh),θh,∇(θh))
    αPrev = αNC∘(sh,ψhPosOrg_in)
    αbPrev = FatiguehistoryVariable∘(αPrev,αhPlusPrev,αbhPlusPrev)
    
    αhPlusPrev = project(αPrev,model,dΩ,order)
    αbhPlusPrev = project(αbPrev,model,dΩ,order)
    
    FdhPrev = FatigueDegrad∘(αbhPlusPrev)
    
    αbPrevVal = evaluate(αbPrev,VectorValue(0.5*L, 2*CH))
    Fdegrad = evaluate(FdhPrev,VectorValue(0.5*L, 2*CH))
    FracEnergyExp = sum(∫((3/8)*((1-sh)/ls + ls*(∇(sh) ⋅ ∇(sh))))*dΩ)
    
    push!(time, t)
    push!(AppDisplacement, vApp)
    push!(NoofCycles, cycle)
    
    push!(αEnergy, αbPrevVal)
    push!(αDegrad, Fdegrad)
    push!(FracEnergy, FracEnergyExp)
    
    Node_Force = sum(∫( n_Γ_Load ⋅ (σ_Bmod∘(ε(uh),ε(uh),sh)) ) *dΓ_Load + ∫( n_Γ_Load ⋅ (σ_Cmod∘(ε_Skw∘(∇(uh),θh),sh) ) )  *dΓ_Load)
    
    push!(Load, Node_Force[2])
    push!(Displacement, vApp)
    if mod(count,10) == 0
         writevtk(Ω,"results_PhaseField_Mode1_N0.5l0.10_$count",cellfields=
        ["uh"=>uh,"s"=>sh ,"θ" => θh, "epsi"=>ε(uh)])
    end
end

In [ ]:
writevtk(Ω,"results__PhaseField_Mode1_N0.5l0.10_$count",cellfields=
        ["uh"=>uh,"s"=>sh ,"θ" => θh, "epsi"=>ε(uh)])

In [ ]:
plot(Displacement,Load*1e-3)

In [ ]:
parent_path = "../PlottingFiles/"     # Go up one level and define the new folder name
mkpath(parent_path)

In [ ]:
cd("../PlottingFiles")

In [ ]:
using DelimitedFiles

In [ ]:
NoofCyclesCSV = writedlm("DInNotch_kf05_Step16NoofCyclesPhlsfullVolDevN0.5_ls1.0_lb$l.csv",  NoofCycles, ',')
FracEnergyCSV = writedlm("DInNotch_kf05_Step16FracEnergyPhlsfullVolDevN0.5_ls1.0_lb$l.csv",  FracEnergy, ',')
LoadEnergyCSV = writedlm("DInNotch_kf05_Step16LoadPhlsfullVolDevN0.5_ls1.0_lb$l.csv",  Load, ',')

In [ ]:
XloccrackCSV = writedlm("DInNotch_kf05_Step16XloccrackPhlsfullVolDevN0.5_ls1.0_lb$l.csv",  Xloccrack, ',')
FdegradCSV = writedlm("DInNotch_kf05_Step16FdegradPhlsfullVolDevN0.5_ls1.0_lb$l.csv",  αDegrad, ',')
αEnergyCSV = writedlm("DInNotch_kf05_Step16AlphaEnergyPhlsfullVolDevN0.5_ls1.0_lb$l.csv",  αEnergy, ',')